In [1]:
import sys
import tqdm
import time
import sklearn
import numpy as np
import pandas as pd
import scipy
import copy
import random
import math
import json
import torch
import torch.nn.functional as F
from load_dataset import load, generate_random_dataset
from classifier import NeuralNetwork, LogisticRegression, SVM
from utils import *
from metrics import *  # include fairness and corresponding derivatives
from scipy import stats
from scipy.stats import rankdata
from sklearn import metrics, preprocessing
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import classification_report
from operator import itemgetter
from torch.autograd import grad
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
torch.use_deterministic_algorithms(True)

In [47]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
@interact
def show_articles_more_than(column ='age', x = 10):
    return X_train_orig.loc [df [column]> x]

interactive(children=(Text(value='age', description='column'), IntSlider(value=10, description='x', max=30, mi…

In [2]:
# ignore all the warnings
import warnings
warnings.filterwarnings('ignore') 

In [49]:
dataset = 'german'
X_train, X_test, y_train, y_test = load(dataset, sample=False)

duplicates = 1
make_duplicates = lambda x, d: pd.concat([x]*d, axis=0).reset_index(drop=True)
X_train = make_duplicates(X_train, duplicates)
X_test = make_duplicates(X_test, duplicates)
y_train = make_duplicates(y_train, duplicates)
y_test = make_duplicates(y_test, duplicates)

**Parametric Model**

In [4]:
X_train_orig = copy.deepcopy(X_train)
X_test_orig = copy.deepcopy(X_test)

# Scale data: regularization penalty default: ‘l2’, ‘lbfgs’ solvers support only l2 penalties. 
# Regularization makes the predictor dependent on the scale of the features.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Loss function** (Log loss for logistic regression)

In [5]:
# clf = NeuralNetwork(input_size=X_train.shape[-1])
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])
num_params = len(convert_grad_to_ndarray(list(clf.parameters())))
if isinstance(clf, LogisticRegression) or isinstance(clf, NeuralNetwork):
    loss_func = lambda model, x, y_true: logistic_loss_torch(model, x, y_true, 0)
elif isinstance(clf, SVM):
    loss_func = svm_loss_torch

**Compute Accuracy** 

In [6]:
def computeAccuracy(y_true, y_pred):
    return np.sum((y_pred>0.5) == y_true)/len(y_pred)

**First-order derivative of loss function at z with respect to model parameters**

In [7]:
def del_L_del_theta_i(model, x, y_true, retain_graph=False):
    loss = loss_func(model, x, y_true)
    w = [ p for p in model.parameters() if p.requires_grad ]
    return grad(loss, w, create_graph=True, retain_graph=retain_graph)

**First-order derivative of $P(y \mid \textbf{x})$ with respect to model parameters**

In [8]:
def del_f_del_theta_i(model, x, retain_graph=False):
    w = [ p for p in model.parameters() if p.requires_grad ]
    return grad(model(torch.FloatTensor(x)), w, retain_graph=retain_graph)

**Stochastic estimation of Hessian vector product (involving del fairness): $H_{\theta}^{-1}v = H_{\theta}^{-1}\nabla_{\theta}f(z, \theta) = v + [I - \nabla_{\theta}^2L(z_{s_j}, \theta^*)]H_{\theta}^{-1}v$**

In [9]:
def hvp(y, w, v):
    ''' Multiply the Hessians of y and w by v.'''
    # First backprop
    first_grads = grad(y, w, retain_graph=True, create_graph=True)

    # Elementwise products
    elemwise_products = 0
    for grad_elem, v_elem in zip(convert_grad_to_tensor(first_grads), v):
        elemwise_products += torch.sum(grad_elem * v_elem)

    # Second backprop
    return_grads = grad(elemwise_products, w, create_graph=True)

    return return_grads

In [10]:
def hessian_one_point(model, x, y):
    x, y = torch.FloatTensor(x), torch.FloatTensor([y])
    loss = loss_func(model, x, y)
    params = [ p for p in model.parameters() if p.requires_grad ]
    first_grads = convert_grad_to_tensor(grad(loss, params, retain_graph=True, create_graph=True))
    hv = np.zeros((len(first_grads), len(first_grads)))
    for i in range(len(first_grads)):
        hv[i, :] = convert_grad_to_ndarray(grad(first_grads[i], params, create_graph=True)).ravel()
    return hv

In [11]:
# Compute multiplication of inverse hessian matrix and vector v
def s_test(model, xs, ys, v, hinv=None, damp=0.01, scale=25.0, r=-1, batch_size=-1, recursive=False, verbose=False):
    ''' Arguments:
        xs: list of data points
        ys: list of true labels corresponding to data points in xs
        damp: dampening factor
        scale: scaling factor
        r: number of iterations aka recursion depth
            should be enough so that the value stabilises.
        batch_size: number of instances in each batch in recursive approximation
        recursive: determine whether to recursively approximate hinv_v'''
    xs, ys = torch.FloatTensor(xs.copy()), torch.FloatTensor(ys.copy())
    n = len(xs)
    if recursive:
        hinv_v = copy.deepcopy(v)
        if verbose:
            print('Computing s_test...')
            tbar = tqdm.tqdm(total=r)
        if (batch_size == -1):  # default
            batch_size = 10
        if (r == -1):
            r = n // batch_size + 1
        sample = np.random.choice(range(n), r*batch_size, replace=True)
        for i in range(r):
            sample_idx = sample[i*batch_size:(i+1)*batch_size]
            x, y = xs[sample_idx], ys[sample_idx]
            loss = loss_func(model, x, y)
            params = [ p for p in model.parameters() if p.requires_grad ]
            hv = convert_grad_to_ndarray(hvp(loss, params, torch.FloatTensor(hinv_v)))
            # Recursively caclulate h_estimate
            hinv_v = v + (1 - damp) * hinv_v - hv / scale
            if verbose:
                tbar.update(1)
    else:
        if hinv is None:
            hinv = np.linalg.pinv(np.sum(hessian_all_points, axis=0))
        scale = 1.0
        hinv_v = np.matmul(hinv, v)

    return hinv_v / scale

**Metrics: Initial state**

In [12]:
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = NeuralNetwork(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])

clf.fit(X_train, y_train, use_sklearn=True)

y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)
print("Initial statistical parity: ", spd_0)

tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1, dataset)
print("Initial TPR parity: ", tpr_parity_0)

predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2, dataset)
print("Initial predictive parity: ", predictive_parity_0)

loss_0 = logistic_loss(y_test, y_pred_test)
print("Initial loss: ", loss_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial statistical parity:  -0.1937291073930064
Initial TPR parity:  -0.180429660523945
Initial predictive parity:  -0.18632730832832872
Initial loss:  0.3978844808353249
Initial accuracy:  0.801394422310757


In [13]:
# hessian_all_points = []
# tbar = tqdm.tqdm(total=len(X_train))
# total_time = 0
# for i in range(len(X_train)):
#     t0 = time.time()
#     hessian_all_points.append(hessian_one_point(clf, X_train[i], y_train[i])/len(X_train))
#     total_time += time.time()-t0
#     tbar.update(1)

In [14]:
# hessian_all_points = np.array(hessian_all_points)

**Pre-compute: del_L_del_theta for each training data point**

In [15]:
del_L_del_theta = []
for i in range(int(len(X_train))):
    gradient = convert_grad_to_ndarray(del_L_del_theta_i(clf, X_train[i], int(y_train[i])))
    while np.sum(np.isnan(gradient))>0:
        gradient = convert_grad_to_ndarray(del_L_del_theta_i(clf, X_train[i], int(y_train[i])))
    del_L_del_theta.append(gradient)
del_L_del_theta = np.array(del_L_del_theta)

*Select delta fairness function depending on selected metric*

In [16]:
metric = 0
if metric == 0:
    v1 = del_spd_del_theta(clf, X_test_orig, X_test, dataset)
elif metric == 1:
    v1 = del_tpr_parity_del_theta(clf, X_test_orig, X_test, y_test, dataset)
elif metric == 2:
    v1 = del_predictive_parity_del_theta(clf, X_test_orig, X_test, y_test, dataset)

**Update**

In [17]:
# def del_L_del_delta_i(num_params, x, y_pred, params, y_true):
#     del_L_del_delta = np.ones((num_params - 1, num_params)) * (y_pred[0] * y_pred[1])
#     for i in range(num_params - 1):
#         for j in range(num_params):
#             del_L_del_delta[i][j] *=  params[i]
#             if j != 0:
#                 del_L_del_delta[i][j] *=  x[j - 1]
#                 if j == i:
#                     del_L_del_delta[i][j] += y_pred[1] - y_true
            
#     return del_L_del_delta

# def del_L_del_delta_i(model, x, delta, y_true, retain_graph=False):
#     loss = loss_func(model, x+delta, y_true)
#     return grad(loss, delta, create_graph=True, retain_graph=retain_graph)
def del_L_del_theta_del_delta_i(model, x, delta, y_true):
    x, delta, y = torch.FloatTensor(x), torch.FloatTensor(delta), torch.FloatTensor([y_true])
    delta.requires_grad = True
    loss = loss_func(model, x+delta, y)
    params = [ p for p in model.parameters() if p.requires_grad ]
    first_grads = convert_grad_to_tensor(grad(loss, params, retain_graph=True, create_graph=True))
    result = np.zeros((x.shape[1], len(first_grads)))
    for i in range(len(first_grads)):
        result[:, i] = convert_grad_to_ndarray(grad(first_grads[i], delta, create_graph=True)).ravel()
    return result

In [18]:
def repair(idx, numIter, learningRate):
    clf.fit(X_train, y_train, use_sklearn=True)
    y_pred_test = clf.predict_proba(X_test)
    
    X_p = copy.deepcopy(X_train[idx])
    y_p_true = copy.deepcopy(y_train[idx]) 

    del_L_del_delta = np.zeros((X_train.shape[1], num_params))
    
#     torch.manual_seed(1)
#     delta_new = torch.rand((X_train.shape[1], 1))
    random.seed(0)
    delta_new = np.zeros((num_params - 1, 1))
    for i in range(len(delta_new)):
        delta_new[i] = random.random()
    delta_new = torch.FloatTensor(delta_new)
    threshold = 0.05
    delta_old = -1 * torch.ones((X_train.shape[1], 1))

    num_iter = 0
    del_L_del_theta_Xp = np.sum(del_L_del_theta[idx, :], axis=0).reshape(-1, 1)
    obj_old = np.dot(del_L_del_theta_Xp.transpose(), v1)
    obj_new = obj_old + 0.1
    print(obj_old, obj_new)

    while (obj_new > obj_old):
        obj_old = obj_new
        num_iter += 1
        del_L_del_delta = del_L_del_theta_del_delta_i(clf, X_train[idx, :], delta_new.ravel(), np.array(y_train[idx]))
        delta_old = delta_new
        delta_new += (learningRate/num_iter) * np.dot(del_L_del_delta, v1).reshape(-1, 1)
        
        print(delta_new, (learningRate/num_iter) * np.dot(del_L_del_delta, v1).reshape(-1, 1))
        X_train_perturbed = copy.deepcopy(X_train)
        X_train_perturbed[idx, :] += delta_new.detach().numpy().squeeze()
            
        del_L_del_theta_Xp = convert_grad_to_ndarray(del_L_del_theta_i(clf, X_train_perturbed[idx, :], np.array(y_train[idx]))).reshape(-1, 1)
        obj_new = np.dot(del_L_del_theta_Xp.transpose(), v1)
        print(obj_old, obj_new)
        if ((obj_new < obj_old)):
            return delta_old
    
    return delta_new

In [37]:
random.seed(0)
clf.fit(X_train, y_train, use_sklearn=True)
delta_new = np.zeros((num_params - 1, 1))
for i in range(len(delta_new)):
    delta_new[i] = random.random()
delta_new = torch.FloatTensor(delta_new)

x, delta, y = X_train[idx[0], :], delta_new.ravel(), np.array(y_train)[idx[0]]
x, y = torch.FloatTensor([x])+torch.FloatTensor(delta), torch.FloatTensor([y])
x.requires_grad = True
loss = loss_func(clf, x, y)
params = [ p for p in clf.parameters() if p.requires_grad ]
first_grads = convert_grad_to_tensor(grad(loss, params, retain_graph=True, create_graph=True))
result = np.zeros((x.shape[1], len(first_grads)))
for i in range(len(first_grads)):
    result[:, i] = convert_grad_to_ndarray(grad(first_grads[i], x, create_graph=True)).ravel()/len(idx)

In [40]:
first_grads.detach().numpy()

array([ 0.88456625,  0.48002872,  0.62960243,  0.03082163, -0.15646912,
        0.30763996, -0.25090474, -0.13656567,  0.38038546], dtype=float32)

In [29]:
first_grads

tensor([ 0.8846,  0.4800,  0.6296,  0.0308, -0.1565,  0.3076, -0.2509, -0.1366,
         0.3804], grad_fn=<CatBackward>)

In [34]:
convert_grad_to_ndarray(grad(first_grads[-1], x, create_graph=True)).ravel()

array([ 0.04982051, -0.01277423,  0.14352342,  0.11147176,  0.1353553 ,
        0.01770828,  0.03118311,  0.07162918], dtype=float32)

In [20]:
idx = X_train_orig[
    (X_train_orig['gender'] == 0)
    & (X_train_orig['relationship'] == 0)
    & (X_train_orig['education'] == 12)
    & (X_train_orig['race'] == 1)
    ].index
# v_pert = repair(idx, numIter, learningRate)
v_pert = repair(idx, 100, 1)

[9.71182258] [9.81182258]
tensor([[0.8458],
        [0.7554],
        [0.4400],
        [0.2452],
        [0.4933],
        [0.4008],
        [0.7296],
        [0.2977]], dtype=torch.float64) [[ 0.00139519]
 [-0.00258672]
 [ 0.0194724 ]
 [-0.01372312]
 [-0.01795746]
 [-0.00417402]
 [-0.05422114]
 [-0.00560575]]
[9.81182258] [0.03442867]


In [38]:
df = copy.deepcopy(X_train_orig)
df['income'] = y_train
df.iloc[idx]

,age,workclass,education,marital,relationship,race,gender,hours,income
90,1,7,12,1,0,1,0,0,0
120,0,7,12,0,0,1,0,1,0
201,1,7,12,1,0,1,0,0,0
285,1,7,12,0,0,1,0,0,0
340,0,7,12,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
29779,1,6,12,0,0,1,0,0,0
30004,0,7,12,1,0,1,0,0,0
30131,0,7,12,0,0,1,0,0,0
30144,1,3,12,1,0,1,0,1,0


In [39]:
X_test_orig

,age,workclass,education,marital,relationship,race,gender,hours
0,0,7,6,0,0,0,1,0
1,0,7,8,2,1,1,1,1
2,0,3,12,2,1,1,1,0
3,0,7,9,2,1,0,1,0
4,0,7,5,0,0,1,1,0
...,...,...,...,...,...,...,...,...
15055,0,7,10,0,0,1,1,0
15056,0,7,10,1,0,1,0,0
15057,0,7,10,2,1,1,1,1
15058,0,7,10,1,0,0,1,0


In [40]:
y_pred_test

array([0.01934698, 0.41743198, 0.5349799 , ..., 0.53971857, 0.08282077,
       0.5580527 ], dtype=float32)

In [42]:
%%time
res_age = []
res_workclass = []
res_education = []
res_marital = []
res_relationship = []
res_race = []
res_gender = []
res_hours = []

X_train_copy = copy.deepcopy(X_train)
numCols = len(X_train_orig.columns)
for ix in range(len(idx)):
    X_train_pert = np.zeros((len(X_train[idx[ix]]), 1))
    for i in range(len(X_train[idx[ix]])):
        X_train_pert[i] = X_train[idx[ix]][i] + v_pert[i]

    x0 = np.random.rand(1,numCols)
    mins = []
    maxs = []
    numCols = len(X_train[0])
    for i in range(numCols):
        mins.insert(i, min(X_train[:,i]))
        maxs.insert(i, max(X_train[:,i]))

    from scipy.optimize import Bounds, minimize
    bounds = Bounds(mins, maxs)

    f = lambda x: np.linalg.norm(x - X_train_pert)

    x0 = np.random.rand(numCols)
    res = minimize(f, x0, method='trust-constr', options={'verbose': 0}, bounds=bounds)
    
    for i in range(len(X_train_copy[idx[ix]])):
        X_train_copy[idx[ix]][i] = res.x[i]
        
#     res_x_inv_transform = sc.inverse_transform(res.x, copy=None)
    res_x_inv_transform = sc.inverse_transform(X_train_copy[idx[ix]], copy=None)
#     res_x_inv_transform = sc.inverse_transform(X_train_copy[idx[ix]], copy=None)
    res_age.insert(ix, res_x_inv_transform[0])
    res_workclass.insert(ix, res_x_inv_transform[1])
    res_education.insert(ix, res_x_inv_transform[2])
    res_marital.insert(ix, res_x_inv_transform[3])
    res_relationship.insert(ix, res_x_inv_transform[4])
    res_race.insert(ix, res_x_inv_transform[5])
    res_gender.insert(ix, res_x_inv_transform[6])
    res_hours.insert(ix, res_x_inv_transform[7])

clf.fit(X_train_copy, y_train, use_sklearn=True)
y_pred_test = clf.predict_proba(X_test)

import statistics
print(statistics.mode(res_age), statistics.mode(res_workclass), statistics.mode(res_education), 
      statistics.mode(res_marital), statistics.mode(res_relationship), 
      statistics.mode(res_race), statistics.mode(res_gender), statistics.mode(res_hours))
print(computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)
      /spd_0 - 1)


0.5857465864596665 6.9999987139550495 10.402278396946304 1.674998151183459 0.7527654848444509 0.9999999276573958 0.9508437383210989 0.5755269090306605
-0.019808927249723607
CPU times: user 59.9 s, sys: 4.16 s, total: 1min 4s
Wall time: 32.3 s


In [ ]:
Initial: [[ 0.32698744  2.20756884  0.52520941  0.26284814 -0.14408554]]

In [26]:
df = pd.read_table('salary.data', names=cols, sep="\t", index_col=False)
df = preprocess(df)
# df[df['status', 'credit'].groupby(by="credit").sum()

In [45]:
clf.fit(X_train_copy, y_train, use_sklearn=True)
y_pred_test = clf.predict_proba(X_test)
computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)

-0.18989370294446486

In [ ]:
computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)